In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd "/content/gdrive/MyDrive/Resume Project/01 Production ML Pipeline with Drift Monitoring, Auto-Retraining & Scalable Deployment"

/content/gdrive/MyDrive/Resume Project/01 Production ML Pipeline with Drift Monitoring, Auto-Retraining & Scalable Deployment


In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import os
import sys
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer


In [ ]:
#Loading the dataset
dataset =pd.read_csv("/content/gdrive/MyDrive/Resume Project/01 Production ML Pipeline with Drift Monitoring, Auto-Retraining & Scalable Deployment/dataset/raw/Telco_customer_churn.csv")

In [ ]:
#Copying the data to preventing the accidental overwrites
telco_dataset = dataset.copy()

### Step1) Dropping irrelevant columns

In [ ]:
#Checking whether categorical column are relevant or not for prediction
categorical_column = telco_dataset.select_dtypes(include="object").columns
for col in categorical_column:
  if col not in ["customerID", "TotalCharges", "Churn"]:
    print("-" * 80)
    print("")
    print(f"Target relationship Test for Churn and {col} is : \n ")
    print(pd.crosstab(telco_dataset[col], telco_dataset['Churn'], normalize="index"))
    print("-" * 80)


--------------------------------------------------------------------------------

Target relationship Test for Churn and gender is : 
 
Churn         No       Yes
gender                    
Female  0.730791  0.269209
Male    0.738397  0.261603
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Target relationship Test for Churn and Partner is : 
 
Churn          No       Yes
Partner                    
No       0.670420  0.329580
Yes      0.803351  0.196649
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Target relationship Test for Churn and Dependents is : 
 
Churn             No       Yes
Dependents                    
No          0.687209  0.312791
Yes         0.845498  0.154502
--------------------------------------------------------------------------------


| Churn-rate spread | Decision           |
| ----------------- | ------------------ |
| < 2%              | ❌ Drop             |
| 2–5%              | ⚠️ Weak (optional) |
| 5–10%             | ✅ Useful           |
| > 10%             | 🔥 Strong          |


**So by analysing table of Churn rate spread of other columns, We would be droping CustomerID, gender from dataset as they are not contributing to target columns.**

In [ ]:
#Dropping Columns [gender and customerID]
telco_dataset.drop(columns=["gender", "customerID"], inplace=True)

In [ ]:
#Looking at top 5 rows of dataset to look whether irrelevant columns are dropped or not
telco_dataset.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


###Step2) Dealing with Null values

From Exploratory data analysis we know that, TotalCharges column has 11 null values/empty values as " ".

So we will be removing those 11 values

In [ ]:
#First of all in order to drop " " , we need to replace it with np.nan
telco_dataset["TotalCharges"] = telco_dataset["TotalCharges"].replace(" ", np.nan)

In [ ]:
#Now dropping them
telco_dataset = telco_dataset.dropna(subset=["TotalCharges"])

In [ ]:
# Now checking for null or empty string values " ".
print((telco_dataset == np.nan).sum())
print((telco_dataset == " ").sum())

SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


### Step3) Converting datatype of TotalCharges column from object to float64

In [ ]:
#Checking for datatype of different columns
telco_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SeniorCitizen     7032 non-null   int64  
 1   Partner           7032 non-null   object 
 2   Dependents        7032 non-null   object 
 3   tenure            7032 non-null   int64  
 4   PhoneService      7032 non-null   object 
 5   MultipleLines     7032 non-null   object 
 6   InternetService   7032 non-null   object 
 7   OnlineSecurity    7032 non-null   object 
 8   OnlineBackup      7032 non-null   object 
 9   DeviceProtection  7032 non-null   object 
 10  TechSupport       7032 non-null   object 
 11  StreamingTV       7032 non-null   object 
 12  StreamingMovies   7032 non-null   object 
 13  Contract          7032 non-null   object 
 14  PaperlessBilling  7032 non-null   object 
 15  PaymentMethod     7032 non-null   object 
 16  MonthlyCharges    7032 non-null   float64
 17  

In [ ]:
#changing the datatypes of columns from astype method
telco_dataset['TotalCharges'] = telco_dataset['TotalCharges'].astype("float64")

In [ ]:
# Checking for the changes
telco_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SeniorCitizen     7032 non-null   int64  
 1   Partner           7032 non-null   object 
 2   Dependents        7032 non-null   object 
 3   tenure            7032 non-null   int64  
 4   PhoneService      7032 non-null   object 
 5   MultipleLines     7032 non-null   object 
 6   InternetService   7032 non-null   object 
 7   OnlineSecurity    7032 non-null   object 
 8   OnlineBackup      7032 non-null   object 
 9   DeviceProtection  7032 non-null   object 
 10  TechSupport       7032 non-null   object 
 11  StreamingTV       7032 non-null   object 
 12  StreamingMovies   7032 non-null   object 
 13  Contract          7032 non-null   object 
 14  PaperlessBilling  7032 non-null   object 
 15  PaymentMethod     7032 non-null   object 
 16  MonthlyCharges    7032 non-null   float64
 17  

### Step4) OneHot Encoding for categorical columns (except Churn column) and standard scaling for numerical columns

In [ ]:
#Separating categorical and numerical columns
categorical_column = telco_dataset.select_dtypes(include = "object").drop(columns="Churn")
numerical_column = telco_dataset.select_dtypes(exclude= "object").columns

In [ ]:
# Creating Transformer for performing scaling and encoding
transformer_instance = ColumnTransformer(
    [
        ("scaler", StandardScaler(), numerical_column),
        ("oh_encoder", OneHotEncoder(handle_unknown="ignore"), categorical_column)
    ]
)


###Preprocessing Strategy

Categorical features will be one-hot encoded and numerical features will be scaled.
Target encoding and application of transformations will be performed during model training.


In [35]:
#Saving the dataset
processed_folder = "processed"
os.makedirs(processed_folder, exist_ok=True)
saving_path = os.path.join( os.getcwd(), f'{os.path.join("dataset", processed_folder)}/telcodata_02_processed.csv')
print(saving_path)
telco_dataset.to_csv(saving_path, index=False)

/content/gdrive/MyDrive/Resume Project/01 Production ML Pipeline with Drift Monitoring, Auto-Retraining & Scalable Deployment/dataset/processed/telcodata_02_processed.csv
